# Data Privacy Final Project
### Bradon Soucy and Sam Will

In [10]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

spotify = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-21/spotify_songs.csv')

In [11]:
print(len(spotify))

32833


In [36]:
spotify = spotify.sort_values(by=['track_artist'])
spotify.head(25)

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
5462,2UjEyfnKzaY7qpBEeESJjv,Even When The Water's Cold,!!!,52,3ywERU9rVKEpCQE50fQOgM,Thr!!!er,2013,Indie Poptimism,21nxmipQe5xtXoHHPaYfnY,pop,...,2,-6.847,0,0.0391,0.10800,0.001280,0.1590,0.866,104.971,227067
3693,7y8aVfDkqt6qirGNivvs0M,One Girl / One Boy,!!!,48,3ywERU9rVKEpCQE50fQOgM,Thr!!!er,2013,GTA V - Radio Mirror Park,31R7Gd07KsDafqJEeL4Hqr,pop,...,10,-5.747,0,0.0407,0.00255,0.000059,0.3220,0.870,117.004,243627
27364,30gdgd5rpm2YYNK88DH2G4,Ring Me Up - Soundsystem Mix,!deladap,5,0ehi67LKSthha5isGMfTQW,Ring Me Up (Soundsystem Mix),2019-01-18,Electro Swing Top 100,5M7xLCQTYJL0lbKIrUn3a4,edm,...,7,-3.360,0,0.0377,0.05370,0.067400,0.0685,0.667,118.977,254213
27304,4OYzBNkbbIdlzdXOx6JjCD,Music Has the Power - Wolfgang Lohr Remix Radi...,!deladap,38,3bmmclWhMybn1nh8leCkKb,Music Has the Power (Wolfgang Lohr Remix),2019-11-22,Electro Swing Top 100,5M7xLCQTYJL0lbKIrUn3a4,edm,...,9,-6.513,0,0.0878,0.07750,0.002600,0.0884,0.769,119.966,172500
27333,5NwWGVjLvbxJZrJLhcRqm1,Music Has the Power - Radio Edit,!deladap,20,5RpBtAjrWqTOKPoHbqWC9z,Music Has the Power (Radio Edit),2019-03-29,Electro Swing Top 100,5M7xLCQTYJL0lbKIrUn3a4,edm,...,7,-4.112,1,0.1070,0.11100,0.000000,0.2300,0.655,104.058,206205
18251,4OYzBNkbbIdlzdXOx6JjCD,Music Has the Power - Wolfgang Lohr Remix Radi...,!deladap,38,3bmmclWhMybn1nh8leCkKb,Music Has the Power (Wolfgang Lohr Remix),2019-11-22,F**KIN PERFECT,3CzQZEf6jmEjDmBIVO9rdR,latin,...,9,-6.513,0,0.0878,0.07750,0.002600,0.0884,0.769,119.966,172500
27393,0CMRCkUdmiHGt7EzEUIskU,Crazy Swing,!deladap,25,7e0TQCD3QxftJWbhiaed7v,I Know What You Want,2012,Electro Swing Top 100,5M7xLCQTYJL0lbKIrUn3a4,edm,...,0,-4.677,0,0.0553,0.13900,0.000033,0.0803,0.821,125.043,223507
22363,5VNIWjz02GSMMYM0kkXdsc,Part Of Me - Bonus Track,"""Dear Evan Hansen"" August 2018 Broadway Cast",39,2M1j4rQyLRsy91VpWTZ56k,Part Of Me (Bonus Track),2018-09-25,urban contemporary,4WiB26kw0INKwbzfb5M6Tv,r&b,...,8,-5.460,0,0.0402,0.61600,0.000000,0.1620,0.168,94.137,306731
18225,6dcdXz3385Tew9tOqpCTCo,Un Poquito,#TocoParaVos,43,4b10EvdYJG27bXYZEACjTy,Un Poquito,2019-08-23,Latin Pop Songs,5uOMV7g6vp8WxoqXfkzKaw,latin,...,11,-3.775,0,0.1680,0.50000,0.000000,0.0887,0.793,92.004,158904
9819,6SN1WX0OVZP2TRMnqfggVx,HIBACHI,$ANFI,14,4BsrreTOIecyV2z0apFumw,Scattered Ideas 2,2019-02-23,Rap Party 24/7 Radio / Gangsta Rap,4sGnz2x5tbHE2YlW1nemfb,rap,...,0,-10.000,1,0.3690,0.28100,0.000007,0.1180,0.683,164.989,128120
